---
# Ataque al Algoritmo de McEliece con códigos Reed-Solomon y Reed-Solomon Generalizados
---

#### Ataque estandar

In [1]:
def codigos_traza(C,C1):

    baseC1cuadrado=[];
    n=C1.generator_matrix().row(0).length();

    for i in range(C1.generator_matrix().nrows()):
        for j in range(i,C1.generator_matrix().nrows()):
            baseC1cuadrado+=[vector(F,[C1.generator_matrix().row(i)[h]*C1.generator_matrix().row(j)[h] for h in range(n)])];
    V = VectorSpace(F,n)

    subV = V.subspace(baseC1cuadrado)
    Gcuadrada = subV.basis_matrix()
    C1cuadrado = codes.LinearCode(Gcuadrada);

    if C1cuadrado.dimension() != (2*C1.dimension()-1):
        raise TypeError('Deben emplearse códigos GRS para realizar el ataque')

    n=len(C1.random_element());
    lista=[]
    for i in range(C.generator_matrix().nrows()):
        for j in range(C1cuadrado.parity_check_matrix().nrows()):
            lista+=[list(vector(F,[F(C.generator_matrix().row(i)[h]*C1cuadrado.parity_check_matrix().row(j)[h]) for h in range(n)]))]

    AAA=matrix(F,lista)
    matrizG_codigo=AAA.right_kernel().basis_matrix()
    C2=codes.LinearCode(matrizG_codigo)

    return [C2, C1]

In [2]:
def reducir_codigos_traza(C,C1):

    while(C1.dimension() > 1):
        [C1,C] = codigos_traza(C,C1);

    return [C1,C]

In [3]:
def ataque_estandar(CCC,CCC1):
    [bb,G]=reducir_codigos_traza(CCC,CCC1)
    bb = bb.generator_matrix()
    G = G.systematic_generator_matrix()

    print("Vector B (formato matrix): ")
    print("filas: " +str(bb.nrows()) + " col: " +str(bb.ncols()))
    show(bb)

    print("Matriz G: ")
    print("filas: " +str(G.nrows()) + " col: " +str(G.ncols()))
    show(G)

    F = CCC.base_ring()

    g = vector(F,[G[0,i] for i in range(G.ncols())])
    b = vector(F,[bb[0,i] for i in range(bb.ncols())])
    a = vector(F,[0 for i in range(bb.ncols())])

    print("Vector a (formato vector): ")
    show(a)

    print("Vector b (formato vector): ")
    show(b)

    print("Vector g (formato vector): ")
    show(g)

    a[0] = 1
    a[1] = 2

    for i in list(range(2,len(g))):
        print("i = " + str(i))
        num = a[1]*b[i]-(a[1]-a[0])*g[i]
        den = b[i]

        a[i] = (num/den)

    return [a,b];

In [4]:
# ------------------------------------------------------------
#
# Funcion que realiza el ataque estándar sin la traza
#
#------------------------------------------------------------

def ataque_estandar(CCC,CCC1):
    [bb,G]=reducir_codigos_traza(CCC,CCC1)
    bb = bb.generator_matrix()
    G = G.systematic_generator_matrix()

    F = CCC.base_ring()

    g = vector(F,[G[0,i] for i in range(G.ncols())])
    b = vector(F,[bb[0,i] for i in range(bb.ncols())])
    a = vector(F,[0 for i in range(bb.ncols())])

    a[0] = 1
    a[1] = 2

    for i in list(range(2,len(g))):
        num = a[1]*b[i]-(a[1]-a[0])*g[i]
        den = b[i]

        a[i] = (num/den)

    return [a,b];

##### Ejemplo ataque estandar

In [5]:
F=GF(11);
a=[F(i) for i in [1,2,9,3,10,8,5,4]];
b=[F(i) for i in [3,10,5,9,9,10,5,2]];
k=5
C=codes.GeneralizedReedSolomonCode(a, k, b);
show(C)
show(C.generator_matrix())

[8, 5, 4] Generalized Reed-Solomon Code over GF(11)

[ 3 10  5  9  9 10  5  2]
[ 3  9  1  5  2  3  3  8]
[ 3  7  9  4  9  2  4 10]
[ 3  3  4  1  2  5  9  7]
[ 3  6  3  3  9  7  1  6]

In [6]:
C1=codes.GeneralizedReedSolomonCode(a, k-1, b);
show(C1)
show(C1.generator_matrix())

[8, 4, 5] Generalized Reed-Solomon Code over GF(11)

[ 3 10  5  9  9 10  5  2]
[ 3  9  1  5  2  3  3  8]
[ 3  7  9  4  9  2  4 10]
[ 3  3  4  1  2  5  9  7]

In [7]:
parametros=ataque_estandar(C,C1)
parametros

[(1, 2, 9, 3, 10, 8, 5, 4), (1, 7, 9, 3, 3, 7, 9, 8)]

In [8]:
parametros

[(1, 2, 9, 3, 10, 8, 5, 4), (1, 7, 9, 3, 3, 7, 9, 8)]

In [9]:
show([F(a1) for a1 in parametros[0]])
show(k)
show([F(b1) for b1 in parametros[1]])

[1, 2, 9, 3, 10, 8, 5, 4]

5

[1, 7, 9, 3, 3, 7, 9, 8]

In [10]:
CCCC = codes.GeneralizedReedSolomonCode([F(a1) for a1 in parametros[0]], k,[ F(b1) for b1 in parametros[1]]);
show(CCCC)
show(CCCC.systematic_generator_matrix())

[8, 5, 4] Generalized Reed-Solomon Code over GF(11)

[1 0 0 0 0 2 2 7]
[0 1 0 0 0 7 8 4]
[0 0 1 0 0 8 9 9]
[0 0 0 1 0 6 7 1]
[0 0 0 0 1 3 1 7]

In [11]:
show(C)
show(C.systematic_generator_matrix())

[8, 5, 4] Generalized Reed-Solomon Code over GF(11)

[1 0 0 0 0 2 2 7]
[0 1 0 0 0 7 8 4]
[0 0 1 0 0 8 9 9]
[0 0 0 1 0 6 7 1]
[0 0 0 0 1 3 1 7]

In [12]:
CCCC.systematic_generator_matrix() == C.systematic_generator_matrix()

True